# 📘 Patient Sentiment Classifier - Deep Learning with DistilBERT

!pip uninstall -y wandb


Installation des bibliothèques nécessaires


In [1]:
!pip install -q transformers datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-

Importation des bibliothèques

In [2]:

import pandas as pd
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset


Importation du fichier de données

In [3]:

# Upload your Excel file manually if running in Colab
from google.colab import files
uploaded = files.upload()


Saving enhanced_patient_feedback_full.xlsx to enhanced_patient_feedback_full.xlsx


Chargement et nettoyage des données

In [4]:

df = pd.read_excel("enhanced_patient_feedback_full.xlsx")
df = df[["complaint_description", "type"]].dropna()
df.head()


,complaint_description,type
0,Excellent follow-up and very human approach.,positive
1,"Friendly staff and comfortable waiting area, a...",positive
2,Huge thanks to the pediatric team for their ki...,positive
3,Nobody came to explain the treatment.,negative
4,Quick service and clear communication from the...,positive


Encodage des étiquettes (labels)

In [5]:

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["type"])


 Séparation du jeu de données (80% entrainement , 20% test )



In [ ]:

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)         


Tokenisation des commentaires

Utilisation du tokenizer `DistilBertTokenizerFast` pour convertir les textes en tokens.  
Les commentaires sont tronqués et paddés pour correspondre au format attendu par le modèle BERT.

In [7]:

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["complaint_description"], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Chargement du modèle DistilBERT pour la classification

In [8]:

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(df["label"].unique()))


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Configuration des paramètres d'entraînement

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none",          # ✅ bloque totalement wandb
    run_name="distilbert-sentiment"  # facultatif : donne un nom fixe
)


Initialisation du `Trainer`

In [14]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)


<ipython-input-14-4593ec5e8e8a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Désactivation complète de l'intégration à WandB pour éviter les erreurs ou les demandes d'authentification lors de l'entraînement.

In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"

import wandb
wandb.init = lambda *args, **kwargs: None  # désactive l'init de wandb


Lancement de l'entraînement du modèle

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.582700,0.473901
2,0.206500,0.211041
3,0.073300,0.173529
4,0.075800,0.180088


TrainOutput(global_step=80, training_loss=0.2729067444801331, metrics={'train_runtime': 170.1105, 'train_samples_per_second': 3.762, 'train_steps_per_second': 0.47, 'total_flos': 9272717905920.0, 'train_loss': 0.2729067444801331, 'epoch': 4.0})

In [17]:

model.save_pretrained("sentiment_model")
tokenizer.save_pretrained("sentiment_model")


('sentiment_model/tokenizer_config.json',
 'sentiment_model/special_tokens_map.json',
 'sentiment_model/vocab.txt',
 'sentiment_model/added_tokens.json',
 'sentiment_model/tokenizer.json')

Fonction permettant de prédire le sentiment (`positive`, `negative`, etc.) d’un commentaire en texte libre en utilisant le modèle entraîné.

In [18]:

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        predicted_label = torch.argmax(probs, dim=1).item()
        sentiment = label_encoder.inverse_transform([predicted_label])[0]
        return sentiment


In [19]:

example = "The nurses were rude and did not explain anything."
print(f"Sentiment: {predict_sentiment(example)}")


Sentiment: negative


In [20]:
def batch_predict(texts):
    return [predict_sentiment(text) for text in texts]

comments = [
    "Everything was clear and well explained.",
    "Waiting time was too long and no one helped.",
    "I had an average experience."
]

results = batch_predict(comments)
for text, sentiment in zip(comments, results):
    print(f"Comment: {text}\nPredicted Sentiment: {sentiment}\n")


Comment: Everything was clear and well explained.
Predicted Sentiment: positive

Comment: Waiting time was too long and no one helped.
Predicted Sentiment: negative

Comment: I had an average experience.
Predicted Sentiment: positive

